<a href="https://colab.research.google.com/github/Nikhitha-S-Pavan/mask_detection_yolov3/blob/main/mask_detection_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/kaggle_dataset/

In [ ]:
!unzip archiveyolo.zip

In [ ]:
cd darknet


In [ ]:
#Configure yolo to use colab resources

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!cat Makefile
!make

In [ ]:
! ./darknet

In [ ]:
!mv /content/drive/MyDrive/kaggle_dataset/yolo /content/darknet/

In [ ]:
cd yolo

In [ ]:
!mv labels/test/*.txt images/test/

In [ ]:
!mv labels/train/*.txt images/train/

In [ ]:
!mv labels/valid/*.txt images/valid/

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo/data/g' test.txt

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo/data/g' train.txt

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo/data/g' valid.txt

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo/data/g' valid.txt

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo\///g' obj.data

In [ ]:
cd ..


In [ ]:
!mv  yolo/images/ data/

In [ ]:
!mv yolo/train.txt /content/darknet/

In [ ]:
!mv yolo/test.txt /content/darknet/

In [ ]:
!mv yolo/valid.txt /content/darknet/

In [ ]:
!mv yolo/obj.data /content/darknet/

In [ ]:
!mv yolo/obj.names /content/darknet/

In [ ]:
pwd

In [ ]:

!sed -i 's/classes=20/classes=2/g' cfg/yolo-voc.2.0.cfg ## change classes=2 with your length of class
!sed -i 's/filters=125/filters=35/g' cfg/yolo-voc.2.0.cfg ## change filters=35 with (classes + 5) * num for me (2 + 5) * 5 = 35 
!cat cfg/yolo-voc.2.0.cfg

In [ ]:
##### train #######
!wget https://pjreddie.com/media/files/darknet19_448.conv.23

In [ ]:
!./darknet detector train obj.data cfg/yolov4.cfg darknet19_448.conv.23 -dont_show 0

In [ ]:
!pip install opencv-contrib-python==3.4.13.47 --force-reinstall

In [ ]:
import numpy as np
import argparse
import imutils
import time
import cv2
import glob
import os
from google.colab.patches import cv2_imshow


labelsPath = "/content/darknet/obj.names"
LABELS = open(labelsPath).read().strip().split("\n")


np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")


weightsPath = "/content/darknet/backup/yolov4_final.weights"
configPath = "/content/darknet/cfg/yolov4.cfg"

print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]


for i in glob.glob("data/images/test/*.jpg")[:5]:
  print(i)
  frame = cv2.imread(i)
  (H, W) = frame.shape[:2]

  # construct a blob from the input frame and then perform a forward
  # pass of the YOLO object detector, giving us our bounding boxes
  # and associated probabilities
  blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
    swapRB=True, crop=False)
  net.setInput(blob)
  start = time.time()
  layerOutputs = net.forward(ln)
  end = time.time()

  # initialize our lists of detected bounding boxes, confidences,
  # and class IDs, respectively
  boxes = []
  confidences = []
  classIDs = []

  # loop over each of the layer outputs
  for output in layerOutputs:
    # loop over each of the detections
    for detection in output:
      # extract the class ID and confidence (i.e., probability)
      # of the current object detection
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]

      # filter out weak predictions by ensuring the detected
      # probability is greater than the minimum probability
      if confidence > 0.5:
        # scale the bounding box coordinates back relative to
        # the size of the image, keeping in mind that YOLO
        # actually returns the center (x, y)-coordinates of
        # the bounding box followed by the boxes' width and
        # height
        box = detection[0:4] * np.array([W, H, W, H])
        (centerX, centerY, width, height) = box.astype("int")

        # use the center (x, y)-coordinates to derive the top
        # and and left corner of the bounding box
        x = int(centerX - (width / 2))
        y = int(centerY - (height / 2))

        # update our list of bounding box coordinates,
        # confidences, and class IDs
        boxes.append([x, y, int(width), int(height)])
        confidences.append(float(confidence))
        classIDs.append(classID)

  # apply non-maxima suppression to suppress weak, overlapping
  # bounding boxes
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,
    0.5)

  # ensure at least one detection exists
  if len(idxs) > 0:
    # loop over the indexes we are keeping
    for i in idxs.flatten():
      # extract the bounding box coordinates
      (x, y) = (boxes[i][0], boxes[i][1])
      (w, h) = (boxes[i][2], boxes[i][3])

      # draw a bounding box rectangle and label on the frame
      color = [int(c) for c in COLORS[classIDs[i]]]
      cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
      text = "{}: {:.4f}".format(LABELS[classIDs[i]],
        confidences[i])
      cv2.putText(frame, text, (x, y - 5),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
  cv2_imshow(frame)
  
    